<h1 align="center"> Segmenting and Clustering Neighborhoods in Toronto </h1>

##### Import Libraries :

In [60]:
import pandas as pd # library for data analsysis.
from bs4 import BeautifulSoup # for pulling data out of HTML and XML files.
import requests # allows you to send HTTP/1.1 requests extremely easily.

##### Scrap List of postal codes of Canada wikipedia page content by using BeautifulSoup package :

In [61]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" # This is a list of postal codes in Canada are located within the city of Toronto in the province of Ontario.
source = requests.get(url).text # Page Source
ca_data = BeautifulSoup(source, 'lxml') # use BeautifulSoup for parsing wiki page by lxml parser. 

In [62]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
# create a new dataframe
toro_df = pd.DataFrame(columns=column_names)

##### Convert content of Postal Code HTML table as dataframe :

In [64]:
content = ca_data.find('div', class_='mw-parser-output') # Select mw-parser-output class from whole page.
postal_code = 0
borough = 0 
neighborhood = 0

table_contents=[]
table=content.find('table') # Select HTML Table
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned': # To ignore not assigned cells in table
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})                                 
toro_df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
toro_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [56]:
toro_df.shape

(103, 3)